In [1]:
import pandas as pd
import numpy as np

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from sentence_transformers import CrossEncoder
    

In [3]:
df=pd.read_csv('fairtrec_alldocs_part1.csv')

In [4]:
df_rel=pd.read_csv('positiverel_v2.csv')
query_id='6'
query_text='Biography/science and academia work group'
df_rel.groupby('query_id').count()

df_rel=df_rel[df_rel.query_id==6]



In [5]:
df_pos_sampled=df[df.doc_id.isin(df_rel.doc_id)].sample(25000, replace=False, random_state=1)
df_pos_sampled.rename(columns={
    'doc_texts':'doc_text'
},inplace=True)
df_pos_sampled=df_pos_sampled[['doc_text','doc_id']]


df_neg_sampled=df[~df.doc_id.isin(df_rel.doc_id)].sample(25000, replace=False, random_state=1)
df_neg_sampled.rename(columns={
    'doc_texts':'doc_text'
},inplace=True)
df_neg_sampled=df_neg_sampled[['doc_text','doc_id']]

df_concat=pd.concat([df_pos_sampled,df_neg_sampled])

In [7]:
df_concat[df_concat.doc_id.isin(df_rel.doc_id)].shape,df_concat[~df_concat.doc_id.isin(df_rel.doc_id)].shape

((25000, 2), (25000, 2))

In [8]:
df_concat.doc_id.nunique()

50000

In [9]:
df_concat['doc_num_id']=df_concat['doc_id'].astype(int)


In [10]:
rel_labels=[]
for idx, row in df_concat.iterrows():
    if row['doc_id'] in df_rel.doc_id.values:
        rel_labels.append(1)
    else:
        rel_labels.append(0)

In [11]:
np.mean(rel_labels)

0.5

In [12]:
rel_labels[-1]

0

In [13]:
df_concat['relevance']=rel_labels
# df_concat.to_csv('fairtrecdata_with_labels_v3.csv',index=False)

In [14]:
# df.loc[df.doc_text.isna(),'doc_text']=''

In [15]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


df=df_concat
df.loc[df.doc_text.isna(),'doc_text']=''
df['doc_id']=df['doc_id'].astype(int)




In [16]:
df_concat.loc[df_concat.doc_text.isna(),'doc_text'].unique()


array([], dtype=object)

In [17]:
model_name='cross-encoder/ms-marco-TinyBERT-L-2-v2'
model = AutoModelForSequenceClassification.from_pretrained(model_name, output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

query_text='Biography/science and academia work group'
pred_arr=[]
doc_id_arr=[]


for idx in range(0, df.shape[0], 32):
    print(idx)
    doc_texts=df.iloc[idx:idx+32].doc_text.values
    doc_ids=df.iloc[idx:idx+32].doc_id.values
    feats=[[query_text, doc_text] for doc_text in doc_texts]
    features = tokenizer(feats,  padding=True, truncation=True, 
                         return_tensors="pt", max_length=100)
    doc_id_arr.extend(doc_ids)

    model.eval()
    with torch.no_grad():
        scores = model(**features).hidden_states[-1][:,0,:]

        pred_arr.append(scores.data.numpy())
        print(scores.shape)


np.save('pred_arr_query_6_v3.npy',np.array(pred_arr))
np.save('doc_id_arr_query_6_v3.npy',np.array(doc_id_arr))



0
torch.Size([32, 128])
32
torch.Size([32, 128])
64
torch.Size([32, 128])
96
torch.Size([32, 128])
128
torch.Size([32, 128])
160
torch.Size([32, 128])
192
torch.Size([32, 128])
224
torch.Size([32, 128])
256
torch.Size([32, 128])
288
torch.Size([32, 128])
320
torch.Size([32, 128])
352
torch.Size([32, 128])
384
torch.Size([32, 128])
416
torch.Size([32, 128])
448
torch.Size([32, 128])
480
torch.Size([32, 128])
512
torch.Size([32, 128])
544
torch.Size([32, 128])
576
torch.Size([32, 128])
608
torch.Size([32, 128])
640
torch.Size([32, 128])
672
torch.Size([32, 128])
704
torch.Size([32, 128])
736
torch.Size([32, 128])
768
torch.Size([32, 128])
800
torch.Size([32, 128])
832
torch.Size([32, 128])
864
torch.Size([32, 128])
896
torch.Size([32, 128])
928
torch.Size([32, 128])
960
torch.Size([32, 128])
992
torch.Size([32, 128])
1024
torch.Size([32, 128])
1056
torch.Size([32, 128])
1088
torch.Size([32, 128])
1120
torch.Size([32, 128])
1152
torch.Size([32, 128])
1184
torch.Size([32, 128])
1216
torch.

torch.Size([32, 128])
9792
torch.Size([32, 128])
9824
torch.Size([32, 128])
9856
torch.Size([32, 128])
9888
torch.Size([32, 128])
9920
torch.Size([32, 128])
9952
torch.Size([32, 128])
9984
torch.Size([32, 128])
10016
torch.Size([32, 128])
10048
torch.Size([32, 128])
10080
torch.Size([32, 128])
10112
torch.Size([32, 128])
10144
torch.Size([32, 128])
10176
torch.Size([32, 128])
10208
torch.Size([32, 128])
10240
torch.Size([32, 128])
10272
torch.Size([32, 128])
10304
torch.Size([32, 128])
10336
torch.Size([32, 128])
10368
torch.Size([32, 128])
10400
torch.Size([32, 128])
10432
torch.Size([32, 128])
10464
torch.Size([32, 128])
10496
torch.Size([32, 128])
10528
torch.Size([32, 128])
10560
torch.Size([32, 128])
10592
torch.Size([32, 128])
10624
torch.Size([32, 128])
10656
torch.Size([32, 128])
10688
torch.Size([32, 128])
10720
torch.Size([32, 128])
10752
torch.Size([32, 128])
10784
torch.Size([32, 128])
10816
torch.Size([32, 128])
10848
torch.Size([32, 128])
10880
torch.Size([32, 128])
10912

torch.Size([32, 128])
19168
torch.Size([32, 128])
19200
torch.Size([32, 128])
19232
torch.Size([32, 128])
19264
torch.Size([32, 128])
19296
torch.Size([32, 128])
19328
torch.Size([32, 128])
19360
torch.Size([32, 128])
19392
torch.Size([32, 128])
19424
torch.Size([32, 128])
19456
torch.Size([32, 128])
19488
torch.Size([32, 128])
19520
torch.Size([32, 128])
19552
torch.Size([32, 128])
19584
torch.Size([32, 128])
19616
torch.Size([32, 128])
19648
torch.Size([32, 128])
19680
torch.Size([32, 128])
19712
torch.Size([32, 128])
19744
torch.Size([32, 128])
19776
torch.Size([32, 128])
19808
torch.Size([32, 128])
19840
torch.Size([32, 128])
19872
torch.Size([32, 128])
19904
torch.Size([32, 128])
19936
torch.Size([32, 128])
19968
torch.Size([32, 128])
20000
torch.Size([32, 128])
20032
torch.Size([32, 128])
20064
torch.Size([32, 128])
20096
torch.Size([32, 128])
20128
torch.Size([32, 128])
20160
torch.Size([32, 128])
20192
torch.Size([32, 128])
20224
torch.Size([32, 128])
20256
torch.Size([32, 128]

torch.Size([32, 128])
28544
torch.Size([32, 128])
28576
torch.Size([32, 128])
28608
torch.Size([32, 128])
28640
torch.Size([32, 128])
28672
torch.Size([32, 128])
28704
torch.Size([32, 128])
28736
torch.Size([32, 128])
28768
torch.Size([32, 128])
28800
torch.Size([32, 128])
28832
torch.Size([32, 128])
28864
torch.Size([32, 128])
28896
torch.Size([32, 128])
28928
torch.Size([32, 128])
28960
torch.Size([32, 128])
28992
torch.Size([32, 128])
29024
torch.Size([32, 128])
29056
torch.Size([32, 128])
29088
torch.Size([32, 128])
29120
torch.Size([32, 128])
29152
torch.Size([32, 128])
29184
torch.Size([32, 128])
29216
torch.Size([32, 128])
29248
torch.Size([32, 128])
29280
torch.Size([32, 128])
29312
torch.Size([32, 128])
29344
torch.Size([32, 128])
29376
torch.Size([32, 128])
29408
torch.Size([32, 128])
29440
torch.Size([32, 128])
29472
torch.Size([32, 128])
29504
torch.Size([32, 128])
29536
torch.Size([32, 128])
29568
torch.Size([32, 128])
29600
torch.Size([32, 128])
29632
torch.Size([32, 128]

torch.Size([32, 128])
37920
torch.Size([32, 128])
37952
torch.Size([32, 128])
37984
torch.Size([32, 128])
38016
torch.Size([32, 128])
38048
torch.Size([32, 128])
38080
torch.Size([32, 128])
38112
torch.Size([32, 128])
38144
torch.Size([32, 128])
38176
torch.Size([32, 128])
38208
torch.Size([32, 128])
38240
torch.Size([32, 128])
38272
torch.Size([32, 128])
38304
torch.Size([32, 128])
38336
torch.Size([32, 128])
38368
torch.Size([32, 128])
38400
torch.Size([32, 128])
38432
torch.Size([32, 128])
38464
torch.Size([32, 128])
38496
torch.Size([32, 128])
38528
torch.Size([32, 128])
38560
torch.Size([32, 128])
38592
torch.Size([32, 128])
38624
torch.Size([32, 128])
38656
torch.Size([32, 128])
38688
torch.Size([32, 128])
38720
torch.Size([32, 128])
38752
torch.Size([32, 128])
38784
torch.Size([32, 128])
38816
torch.Size([32, 128])
38848
torch.Size([32, 128])
38880
torch.Size([32, 128])
38912
torch.Size([32, 128])
38944
torch.Size([32, 128])
38976
torch.Size([32, 128])
39008
torch.Size([32, 128]

torch.Size([32, 128])
47296
torch.Size([32, 128])
47328
torch.Size([32, 128])
47360
torch.Size([32, 128])
47392
torch.Size([32, 128])
47424
torch.Size([32, 128])
47456
torch.Size([32, 128])
47488
torch.Size([32, 128])
47520
torch.Size([32, 128])
47552
torch.Size([32, 128])
47584
torch.Size([32, 128])
47616
torch.Size([32, 128])
47648
torch.Size([32, 128])
47680
torch.Size([32, 128])
47712
torch.Size([32, 128])
47744
torch.Size([32, 128])
47776
torch.Size([32, 128])
47808
torch.Size([32, 128])
47840
torch.Size([32, 128])
47872
torch.Size([32, 128])
47904
torch.Size([32, 128])
47936
torch.Size([32, 128])
47968
torch.Size([32, 128])
48000
torch.Size([32, 128])
48032
torch.Size([32, 128])
48064
torch.Size([32, 128])
48096
torch.Size([32, 128])
48128
torch.Size([32, 128])
48160
torch.Size([32, 128])
48192
torch.Size([32, 128])
48224
torch.Size([32, 128])
48256
torch.Size([32, 128])
48288
torch.Size([32, 128])
48320
torch.Size([32, 128])
48352
torch.Size([32, 128])
48384
torch.Size([32, 128]